In [ ]:
import torch
from torch import nn
from d2l import torch as d2l


In [2]:
# vgg块
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    # 添加卷积层
    for _ in range(num_convs):
        layers.append(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        ) # w-3+1+2 = w 
        layers.append(
            nn.ReLU()
        )
        in_channels = out_channels # 除了第一个卷积层，剩下的输入输出通道相同
    # 添加池化层
    layers.append(
        nn.MaxPool2d(kernel_size=2,stride=2) 
    ) # w/2

    # *layers表示把列表里面的元素按顺序作为参数传入函数
    return nn.Sequential(*layers)    


In [3]:
# 由给定架构构建vgg网络
conv_arch = ((1,64),(1,128),(2,256),(2,512),(2,512))  # (卷积层数，输出通道数)

def vgg(conv_arch):
    conv_blk = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blk.append(
            vgg_block(num_convs, in_channels, out_channels)
        )
        in_channels = out_channels

    # 224 / (2^5) = 7 （卷积层不改变大小，5个池化层每个除2）
    return nn.Sequential(
        *conv_blk, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096, 4096), nn.ReLU(),
        nn.Dropout(0.5), nn.Linear(4096, 10)
    ) 

net = vgg(conv_arch)

In [10]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'Output shape:\t', X.shape)

Sequential Output shape:	 torch.Size([1, 64, 112, 112])
Sequential Output shape:	 torch.Size([1, 128, 56, 56])
Sequential Output shape:	 torch.Size([1, 256, 28, 28])
Sequential Output shape:	 torch.Size([1, 512, 14, 14])
Sequential Output shape:	 torch.Size([1, 512, 7, 7])
Flatten Output shape:	 torch.Size([1, 25088])
Linear Output shape:	 torch.Size([1, 4096])
ReLU Output shape:	 torch.Size([1, 4096])
Dropout Output shape:	 torch.Size([1, 4096])
Linear Output shape:	 torch.Size([1, 4096])
ReLU Output shape:	 torch.Size([1, 4096])
Dropout Output shape:	 torch.Size([1, 4096])
Linear Output shape:	 torch.Size([1, 10])


In [ ]:
# 通道数除4，构建一个较小的VGG（计算量约减小了16倍， ci*co*kh*hw*h*w, 中间的ci和co都减小了四倍）
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [ ]:
from my_model import my_train

In [ ]:
lr, num_epochs, batch_size = 0.001, 10, 128
train_iter, test_iter = train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size,resize=224)   
my_train(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu(), stratergy="Adam")